# 8.데이터 준비하기:조인, 병합, 변형

## 8.1 계층적 색인

In [1]:
import pandas as pd
import numpy as np

data = pd.Series(np.random.randn(9), index = [['a','a','a','b','b','c','c','d','d'],[1,2,3,1,3,1,2,2,3]])

data

a  1   -0.779384
   2   -0.321246
   3    0.853799
b  1    0.043900
   3   -2.293148
c  1   -1.073694
   2   -2.163342
d  2    1.164405
   3    1.591980
dtype: float64

In [2]:
data.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

색인이 두 개라 MultiIndex

In [3]:
data['b']

1    0.043900
3   -2.293148
dtype: float64

In [4]:
data['b':'c']

b  1    0.043900
   3   -2.293148
c  1   -1.073694
   2   -2.163342
dtype: float64

In [8]:
data.loc[['b','d']]

b  1    0.043900
   3   -2.293148
d  2    1.164405
   3    1.591980
dtype: float64

In [9]:
data.loc[:,2]

a   -0.321246
c   -2.163342
d    1.164405
dtype: float64

In [11]:
data.unstack()

,1,2,3
a,-0.779384,-0.321246,0.853799
b,0.043900,NaN,-2.293148
c,-1.073694,-2.163342,NaN
d,NaN,1.164405,1.591980


In [12]:
data.unstack().stack()

a  1   -0.779384
   2   -0.321246
   3    0.853799
b  1    0.043900
   3   -2.293148
c  1   -1.073694
   2   -2.163342
d  2    1.164405
   3    1.591980
dtype: float64

In [14]:
frame = pd.DataFrame(np.arange(12).reshape((4,3)),index = [['a','a','b','b'],[1,2,1,2]], 
                     columns = [['Ohio','Ohio','Colorado'],['Green','Red','Green']])

frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

In [15]:
frame.stack()

Colorado  Ohio
a 1 Green       2.0     0
    Red         NaN     1
  2 Green       5.0     3
    Red         NaN     4
b 1 Green       8.0     6
    Red         NaN     7
  2 Green      11.0     9
    Red         NaN    10

In [16]:
frame.index.names = ['key1','key2']
frame.columns.names = ['state','color']

frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [17]:
frame['Ohio']

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

### 8.1.1 계층의 순서를 바꾸고 정렬하기

In [18]:
frame.swaplevel('key1','key2')

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

In [21]:
frame.sort_index(level=1)

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

In [22]:
frame.swaplevel(0,1).sort_index(level=0)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

### 8.1.2 계층별 요약 통계

In [23]:
frame.sum(level='key2')

state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [25]:
frame.sum(level='color',axis=1)

color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

### 8.1.3 DataFrame의 컬럼 사용하기

In [26]:
frame = pd.DataFrame({'a':range(7),'b':range(7,0,-1),
                      'c':['one','one','one','two','two','two','two'], 'd':[0,1,2,0,1,2,3]})
frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


In [27]:
frame2 = frame.set_index(['c','d'])
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

In [28]:
frame.set_index(['c','d'],drop=False)

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

In [29]:
frame2.reset_index()

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


## 8.2 데이터 합치기

### 8.2.1 데이터베이스 스타일로 DataFrame 합치기

In [30]:
df1 = pd.DataFrame({'key':['b','b','a','c','a','a','b'],
                    'data1':range(7)})

df2 = pd.DataFrame({'key':['a','b','d'],
                    'data2':range(3)})
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [31]:
df2

,key,data2
0,a,0
1,b,1
2,d,2


In [32]:
pd.merge(df1,df2)

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [33]:
pd.merge(df1,df2,on='key')

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [36]:
df3 = pd.DataFrame({'lkey':['b','b','a','c','a','a','b'],
                    'data1':range(7)})

df4 = pd.DataFrame({'rkey':['a','b','d'],
                    'data2':range(3)})

df3

,lkey,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [37]:
df4

,rkey,data2
0,a,0
1,b,1
2,d,2


In [39]:
pd.merge(df3,df4,left_on = 'lkey',right_on = 'rkey')

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


In [40]:
pd.merge(df1,df2,how='outer')

,key,data1,data2
0,b,0.0,1.0
1,b,1.0,1.0
2,b,6.0,1.0
3,a,2.0,0.0
4,a,4.0,0.0
5,a,5.0,0.0
6,c,3.0,NaN
7,d,NaN,2.0


In [41]:
left = pd.DataFrame({'key1':['foo','foo','bar'],
                     'key2':['one','two','one'],
                     'lval':[1,2,3]})

right = pd.DataFrame({'key1':['foo','foo','bar','bar'],
                      'key2':['one','one','one','two'],
                      'rval':[4,5,6,7]})

pd.merge(left,right, on=['key1','key2'], how='outer')

,key1,key2,lval,rval
0,foo,one,1.0,4.0
1,foo,one,1.0,5.0
2,foo,two,2.0,NaN
3,bar,one,3.0,6.0
4,bar,two,NaN,7.0


In [42]:
pd.merge(left, right, on = 'key1')

,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [43]:
pd.merge(left,right, on = 'key1', suffixes = ('_left','_right'))

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


### 8.2.2 색인 병합하기

In [45]:
left1 = pd.DataFrame({'key':['a','b','a','a','b','c'], 
                      'value':range(6)})
right1 = pd.DataFrame({'group_val':[3.5,7]},index=['a','b'])

left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [46]:
right1

,group_val
a,3.5
b,7.0


In [51]:
pd.merge(left1, right1, left_on = 'key', right_index = True)

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


In [52]:
pd.merge(left1,right1, left_on = 'key', right_index = True, how = 'outer')

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


In [53]:
left2 = pd.DataFrame([[1,2],[3,4],[5,6]],
                     index=['a','c','e'],
                     columns=['Ohio','Nevada'])
right2 = pd.DataFrame([[7,8],[9,10],[11,12],[13,14]],
                      index=['b','c','d','e'],
                      columns = ['Missouri','Alabama'])
left2

,Ohio,Nevada
a,1,2
c,3,4
e,5,6


In [54]:
right2

,Missouri,Alabama
b,7,8
c,9,10
d,11,12
e,13,14


In [55]:
pd.merge(left2,right2,how='outer',left_index=True,right_index=True)

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


In [57]:
left2.join(right2,how='outer')

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


### 8.2.3 축 따라 이어붙이기

In [58]:
arr = np.arange(12).reshape((3,4))

arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [59]:
np.concatenate([arr,arr],axis=1)

array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

In [60]:
s1 = pd.Series([0,1], index = ['a','b'])
s2 = pd.Series([2,3,4], index = ['c','d','e'])
s3 = pd.Series([5,6], index = ['f','g'])

In [61]:
pd.concat([s1,s2,s3])

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

In [62]:
pd.concat([s1,s2,s3], axis=1)

C:\Users\jinyoung\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [63]:
s4 = pd.concat([s1,s3])
s4

a    0
b    1
f    5
g    6
dtype: int64

In [64]:
pd.concat([s1,s4],axis = 1)

C:\Users\jinyoung\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,0,1
a,0.0,0
b,1.0,1
f,NaN,5
g,NaN,6


In [65]:
pd.concat([s1,s4],axis=1,join='inner')

,0,1
a,0,0
b,1,1


In [67]:
pd.concat([s1,s4],axis=1,join_axes=[['a','c','b','e']])

C:\Users\jinyoung\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  """Entry point for launching an IPython kernel.


,0,1
a,0.0,0.0
c,NaN,NaN
b,1.0,1.0
e,NaN,NaN


병합하려는 축을 지정 가능

In [68]:
result = pd.concat([s1,s2,s3],keys=['one','two','three'])
result

one    a    0
       b    1
two    c    2
       d    3
       e    4
three  f    5
       g    6
dtype: int64

계층적 색인으로 구분할 수 있음

In [71]:
result.index

MultiIndex([(  'one', 'a'),
            (  'one', 'b'),
            (  'two', 'c'),
            (  'two', 'd'),
            (  'two', 'e'),
            ('three', 'f'),
            ('three', 'g')],
           )

In [70]:
result.unstack()

,a,b,c,d,e,f,g
one,0.0,1.0,NaN,NaN,NaN,NaN,NaN
two,NaN,NaN,2.0,3.0,4.0,NaN,NaN
three,NaN,NaN,NaN,NaN,NaN,5.0,6.0


In [72]:
pd.concat([s1,s2,s3],axis=1,keys=['one','two','three'])

C:\Users\jinyoung\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,one,two,three
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [73]:
df1 = pd.DataFrame(np.random.randn(3,4), columns=['a','b','c','d'])
df2 = pd.DataFrame(np.random.randn(2,3), columns=['b','d','a'])

df1

,a,b,c,d
0,-1.275127,0.364218,0.671649,1.710367
1,1.054212,0.233717,-1.931018,1.586455
2,0.183828,0.483847,2.185868,0.624122


In [74]:
df2

,b,d,a
0,0.442667,-0.028743,0.615764
1,0.252200,-0.767076,-0.679637


In [82]:
pd.concat([df1,df2], ignore_index=True)

C:\Users\jinyoung\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,a,b,c,d
0,-1.275127,0.364218,0.671649,1.710367
1,1.054212,0.233717,-1.931018,1.586455
2,0.183828,0.483847,2.185868,0.624122
3,0.615764,0.442667,NaN,-0.028743
4,-0.679637,0.252200,NaN,-0.767076


### 8.2.4 겹치는 데이터 합치기

In [85]:
df1 = pd.DataFrame({'a':[1.,np.nan,5.,np.nan],
                    'b':[np.nan,2.,np.nan,6.],
                    
                    'c':range(2,18,4)})

df2 = pd.DataFrame({'a':[5.,4.,np.nan,3.,7.],
                    'b':[np.nan,3.,4.,6.,8.]})

df1

,a,b,c
0,1.0,NaN,2
1,NaN,2.0,6
2,5.0,NaN,10
3,NaN,6.0,14


In [84]:
df2

,b,d,a
0,0.442667,-0.028743,0.615764
1,0.252200,-0.767076,-0.679637


In [86]:
df1.combine_first(df2)

,a,b,c
0,1.0,NaN,2.0
1,4.0,2.0,6.0
2,5.0,4.0,10.0
3,3.0,6.0,14.0
4,7.0,8.0,NaN


빈 값 채워줌

## 8.3 재형성과 피벗

### 8.3.1 계층적 색인으로 재형성하기

In [87]:
data = pd.DataFrame(np.arange(6).reshape((2,3)),
                    index=pd.Index(['Ohio','Colorado'], name='state'),
                    columns=pd.Index(['one','two','three'], name='number'))
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [88]:
result = data.stack()
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int32

In [89]:
result.unstack()

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [90]:
result.unstack(0)

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [91]:
result.unstack('state')

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


### 8.3.2 긴 형식에서 넓은 형식으로 피벗하기

In [95]:
ldata = pd.DataFrame({'date':['1959-03-31','1959-03-31','1959-03-31','1959-06-30'],
                      'item':['realgdp','infl','unemp','realgdp'],
                      'value':[12,32423,123,4345]})
ldata

,date,item,value
0,1959-03-31,realgdp,12
1,1959-03-31,infl,32423
2,1959-03-31,unemp,123
3,1959-06-30,realgdp,4345


In [96]:
ldata.pivot('date','item','value')

item,infl,realgdp,unemp
date,,,
1959-03-31,32423.0,12.0,123.0
1959-06-30,NaN,4345.0,NaN


### 8.3.3 넓은 형식에서 긴 형식으로 피벗하기

In [97]:
df = pd.DataFrame({'key':['foo','bar','baz'],
                   'A':[1,2,3],
                   'B':[4,5,6],
                   'C':[7,8,9]})

df

,key,A,B,C
0,foo,1,4,7
1,bar,2,5,8
2,baz,3,6,9


In [99]:
melted = pd.melt(df,['key'])
melted

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6
6,foo,C,7
7,bar,C,8
8,baz,C,9


피벗 반대

In [103]:
reshaped= melted.pivot('key','variable','value')
reshaped

variable,A,B,C
key,,,
bar,2,5,8
baz,3,6,9
foo,1,4,7


In [104]:
reshaped.reset_index()

variable,key,A,B,C
0,bar,2,5,8
1,baz,3,6,9
2,foo,1,4,7


In [105]:
pd.melt(df, id_vars=['key'], value_vars=['A','B'])

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6


지정해서 녹일수도 있음

In [106]:
pd.melt(df, value_vars = ['key','A','B'])

,variable,value
0,key,foo
1,key,bar
2,key,baz
3,A,1
4,A,2
5,A,3
6,B,4
7,B,5
8,B,6
